In [4]:
import pandas as pd

# CSV dosyalarını okuma
df1 = pd.read_csv('inme-durumu-test-data.csv')
df2 = pd.read_csv('inme-durumu-cluster-test-data.csv')

# 'filename' ve 'image-name' sütunları üzerinden birleştirme
# Önce, her iki df üzerinde 'filename' ve 'image-name' sütunlarının eşleşip eşleşmediğini kontrol ederiz

# Birleştirilecek sütunları eşleştir
df_merged = pd.merge(df1, df2, left_on='image-name', right_on='filename', how='left')
df_merged


,image-name,inme-durumu,filename,cluster,section
0,10007.png,1,10007.png,3,D
1,10020.png,0,10020.png,1,B
2,10029.png,1,10029.png,1,B
3,10064.png,0,10064.png,0,A
4,10110.png,0,10110.png,3,D
...,...,...,...,...,...
292,16969.png,1,16969.png,2,C
293,17002.png,1,17002.png,0,A
294,17014.png,1,17014.png,1,B
295,17017.png,1,17017.png,1,B


In [5]:
columns_to_drop = ['cluster', 'image-name','inme-durumu']  # Bu sütunları kendinize göre değiştirin
df_merged = df_merged.drop(columns=columns_to_drop)
df_merged

,filename,section
0,10007.png,D
1,10020.png,B
2,10029.png,B
3,10064.png,A
4,10110.png,D
...,...,...
292,16969.png,C
293,17002.png,A
294,17014.png,B
295,17017.png,B


In [6]:
df_merged = df_merged.rename(columns={
    'filename': 'img_name'  # Örnek: 'filename' -> 'dosya_adi'
    # Diğer sütun adlarını buraya ekleyebilirsiniz
})

df_merged.to_csv('test-data-name-class.csv', index=False)


In [ ]:

# Sonuçları gösterme
print(df_merged)

# Yeni DataFrame'i bir CSV dosyasına kaydetmek isterseniz:
df_merged.to_csv('birlesmis_dosya.csv', index=False)

TRAİN AND TEST DATA SPLİT

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

# CSV dosyasını yükleme
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# 1. Yöntem: Rastgele Bölme
def random_split(df, test_size=0.3, random_state=42):
    return train_test_split(df, test_size=test_size, random_state=random_state)

# 2. Yöntem: Stratified Split (Label dengesini koruyarak)
def stratified_split(df, test_size=0.3, random_state=42):
    return train_test_split(df, test_size=test_size, random_state=random_state, stratify=df['label'])

# 3. Yöntem: Section Bazlı Ayrım
def section_based_split(df, train_frac=0.7, random_state=42):
    train = df.groupby('section', group_keys=False).apply(lambda x: x.sample(frac=train_frac, random_state=random_state))
    test = df.drop(train.index)
    return train, test

# 4. Yöntem: Dosya İsmi Bazlı Bölme (Eğer img_name sıralı bir yapıdaysa)
def sorted_split(df, train_frac=0.7):
    df_sorted = df.sort_values(by="img_name")
    split_idx = int(len(df_sorted) * train_frac)
    return df_sorted.iloc[:split_idx], df_sorted.iloc[split_idx:]

# 5. Yöntem: K-Fold (İlk Fold'u train-test olarak ayırıyoruz)
def kfold_split(df, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    train_idx, test_idx = next(kf.split(df))
    return df.iloc[train_idx], df.iloc[test_idx]

# Ana fonksiyon
def main():
    file_path = "final-image-state.csv"  # Dosya adını gerektiği gibi değiştirin
    output_dir = "train_test_splits"  # Çıktı klasörü
    os.makedirs(output_dir, exist_ok=True)
    
    df = load_data(file_path)

    splits = {
        "random": random_split(df),
        "stratified": stratified_split(df),
        "section_based": section_based_split(df),
        "sorted": sorted_split(df),
        "kfold": kfold_split(df)
    }

    for method, (train, test) in splits.items():
        train.to_csv(os.path.join(output_dir, f"train_{method}.csv"), index=False)
        test.to_csv(os.path.join(output_dir, f"test_{method}.csv"), index=False)
        print(f"{method} yöntemiyle veri ayrıldı ve CSV dosyaları {output_dir} klasörüne kaydedildi.")

if __name__ == "__main__":
    main()

random yöntemiyle veri ayrıldı ve CSV dosyaları train_test_splits klasörüne kaydedildi.
stratified yöntemiyle veri ayrıldı ve CSV dosyaları train_test_splits klasörüne kaydedildi.
section_based yöntemiyle veri ayrıldı ve CSV dosyaları train_test_splits klasörüne kaydedildi.
sorted yöntemiyle veri ayrıldı ve CSV dosyaları train_test_splits klasörüne kaydedildi.
kfold yöntemiyle veri ayrıldı ve CSV dosyaları train_test_splits klasörüne kaydedildi.


C:\Users\Gaffar.DESKTOP-LPUGT72\AppData\Local\Temp\ipykernel_15172\483518376.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = df.groupby('section', group_keys=False).apply(lambda x: x.sample(frac=train_frac, random_state=random_state))


In [5]:
import pandas as pd

# CSV dosyalarını okuma
df1 = pd.read_csv('results.csv')
df2 = pd.read_csv('predictions.csv')

# 'filename' ve 'image-name' sütunları üzerinden birleştirme
# Önce, her iki df üzerinde 'filename' ve 'image-name' sütunlarının eşleşip eşleşmediğini kontrol ederiz

# Birleştirilecek sütunları eşleştir
df_merged = pd.merge(df1, df2, left_on='image-name', right_on='img_name', how='left')
df_merged


,image-name,mask,img_name,section,predicted_label
0,10007.png,1,10007.png,3.0,1.0
1,10020.png,1,10020.png,1.0,0.0
2,10029.png,1,10029.png,1.0,1.0
3,10064.png,1,10064.png,0.0,0.0
4,10110.png,1,10110.png,3.0,1.0
...,...,...,...,...,...
295,16969.png,0,16969.png,2.0,0.0
296,17002.png,0,17002.png,0.0,0.0
297,17014.png,1,17014.png,1.0,0.0
298,17017.png,1,17017.png,1.0,0.0


In [6]:
df_merged

,image-name,mask,img_name,section,predicted_label
0,10007.png,1,10007.png,3.0,1.0
1,10020.png,1,10020.png,1.0,0.0
2,10029.png,1,10029.png,1.0,1.0
3,10064.png,1,10064.png,0.0,0.0
4,10110.png,1,10110.png,3.0,1.0
...,...,...,...,...,...
295,16969.png,0,16969.png,2.0,0.0
296,17002.png,0,17002.png,0.0,0.0
297,17014.png,1,17014.png,1.0,0.0
298,17017.png,1,17017.png,1.0,0.0


In [8]:
import numpy as np
import pandas as pd

# Sadece mask ve label sütunları için geçerli (finite) olan satırları tut
df_merged = df_merged[np.isfinite(df_merged['mask']) & np.isfinite(df_merged['predicted_label'])]

df_merged['mask'] = df_merged['mask'].astype(int)
df_merged['predicted_label'] = df_merged['predicted_label'].astype(int)

# Toplam satır sayısı
total = len(df_merged)

# Aynı değerlere sahip olan satır sayısı
same = (df_merged['predicted_label'] == df_merged['mask']).sum()

# Farklı değerlere sahip olan satır sayısı
different = total - same

# Yüzdeleri hesapla
same_percent = 100 * same / total
different_percent = 100 * different / total

# Sonuçları yazdır
print(f"Aynı değer sayısı: {same} ({same_percent:.2f}%)")
print(f"Farklı değer sayısı: {different} ({different_percent:.2f}%)")


Aynı değer sayısı: 223 (75.08%)
Farklı değer sayısı: 74 (24.92%)


C:\Users\Gaffar.DESKTOP-LPUGT72\AppData\Local\Temp\ipykernel_3736\2164470185.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['mask'] = df_merged['mask'].astype(int)
C:\Users\Gaffar.DESKTOP-LPUGT72\AppData\Local\Temp\ipykernel_3736\2164470185.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['predicted_label'] = df_merged['predicted_label'].astype(int)


In [4]:
# Farklı olan satırları filtrele
diff_rows = df_merged[df_merged['label'] != df_merged['mask']]

# Sonuçları yazdır
print("Farklı olan satırlar:")
print(diff_rows)


Farklı olan satırlar:
    image-name  mask   img_name section  label
3    10064.png     1  10064.png       A      0
23   10583.png     1  10583.png       A      0
48   11074.png     1  11074.png       B      0
49   11104.png     1  11104.png       C      0
61   11345.png     1  11345.png       A      0
77   11669.png     0  11669.png       B      1
79   11732.png     1  11732.png       A      0
98   12444.png     1  12444.png       A      0
125  13039.png     1  13039.png       C      0
128  13065.png     1  13065.png       A      0
129  13087.png     0  13087.png       B      1
136  13230.png     1  13230.png       B      0
144  13446.png     1  13446.png       A      0
145  13447.png     1  13447.png       D      0
165  13893.png     1  13893.png       D      0
174  14183.png     0  14183.png       C      1
190  14566.png     1  14566.png       A      0
197  14738.png     1  14738.png       A      0
222  15405.png     0  15405.png       A      1
223  15464.png     1  15464.png       